FCNUM.py checks which CCD is the one in ext. 1 
It then uses CCD (UVIS) 1 to determine groups of images
The images get the keywords MAINCCD in the 0.header

In [ ]:
def get_opt():
    parser=argparse.ArgumentParser()
    parser.add_argument('-radius', type=float, default=None, help='defines max radius for Fieldcenter search')
    parser.add_argument('-onlyshow', action='store_true', default=False, help='give to not rename files but to see what it would be doing')
    parser.add_argument('-survey', default=None, type=str, help='set surveyname')
    parser.add_argument('-dir', type=str, default='./', help='rootdir')
    parser.add_argument('-f', action='store_true', default=False, help='disable interactivity')

    args=parser.parse_args()
    return(args)

Module import and Initialisation
os, fits for file handeling
argparse for setting parameters
np, math for calculations
pyplot or plt for plotting

In [ ]:
def rename(direc, filenames, scaling=0.04, radius=None, onlyshow=False):
    ra_u=[]
    dec_u=[]
    if radius!=None: givenrad=True
    else: givenrad=False

Initialisation, checks if the radius has to be determined

In [ ]:
    for el in filenames:
        #open file, read header
        print '\n\n>>> %s' %(el)
        hdulist=fits.open(direc+el)
        try:
            CCD_1=hdulist[1].header['CCDCHIP']
        except KeyError:
            print '%s does not have information on CCD in its header (ext 0)' %(el)
            continue
        if CCD_1==1:
            print 'ext. 1 is UVIS1'
            refrael=hdulist[1].header['CRVAL1'] 
            refdecel=hdulist[1].header['CRVAL2'] 
            NAXIS2=hdulist[1].header['NAXIS2']
            mainccd=1
        elif CCD_1==2:
            print 'ext. 1 is UVIS2 thus using ext 4'
            refrael=hdulist[4].header['CRVAL1'] 
            refdecel=hdulist[4].header['CRVAL2']
            NAXIS2=hdulist[4].header['NAXIS2']
            mainccd=2
        else:
            print '%s is no flc/flt image, CCD=%s' %(el, CCD_1)
            continue
            

For every file the CCD in ext 1 is determined, if it is 2 ext 4 is used to find CCD 1
The central point refrael refdecel is determined by looking at the header information
Images which fail to provide this information are skipped

In [ ]:
        if not givenrad:
            try:
                radius=scaling*NAXIS2
                print 'search radius is %f'%(radius)
            except:
                radius=0.001
                print 'couldnt find radius, set by default: %f'%(radius)
        else:
            print 'using radius',radius

If the radius is not defined we use the legth of the shorter side of CCD 1 in degree

In [ ]:
        target=hdulist[0].header['TARGNAME'] 
        #assign field center number to image
        c=0
        field_nr=None
        for ra, dec in zip(ra_u, dec_u):
            ra_check=np.mean(ra)
            dec_check=np.mean(dec)

            if check(refrael, ra_check, refdecel, dec_check, radius):
                field_nr=c
                ra_u[c].extend([refrael])
                dec_u[c].extend([refdecel])
                break
            else:
                c+=1
        if field_nr is None:
            field_nr=c
            ra_u.extend([[refrael]])
            dec_u.extend([[refdecel]])
        new_field=change_name(target, field_nr)

field_nr (future FCNUM) is determined, if it already exists the image is assigned to the fieldcenter and the center is added to ra_u[field number] and dec_u[field number], which is further used to correct the field center (by taking the mean of all centers belonging to the fleid) for the next images
if the image does not fit any field a new one is created

In [ ]:
        #fits.setval(el, 'TARGNAME', value=new_field)
        #fits.setval(el, 'TARGNAME', value='30-DOR')
        #print 'new name for value "TARGNAME" %s is %s' %(el, new_field)

        print 'new name for value "FCNUM" (field nr) for %s is %s' %(el, field_nr)


        #splitting files int to new files (UVIS 1 and UVIS 2 in ext.1)

        #newname1='UVIS{}_'.format(mainccd)+el
        newname1=el
        if onlyshow:
            print 'Without -onlyshow: \n\tsave %s, assigning MAINCCD to %s, FCNUM to %s'%(newname1, mainccd, field_nr)
        else:
            hdulist.writeto(newname1, clobber=True)
            fits.setval(newname1, 'FCNUM', value=field_nr)
            fits.setval(newname1, 'MAINCCD', value=str(mainccd))#CCD for image in extension 1 (UVIS)
            print 'save %s, assigning MAINCCD to %s, FCNUM to %s'%(newname1, mainccd, field_nr)
            

If -onlyshow is given the files are not changed, otherwise MAINCCD and FCNUM are assigned to the element

In [ ]:
        """
        bu=hdulist

        if mainccd==1: mainccd_2=2
        elif mainccd==2: mainccd_2=1
        else: print 'Not existing mode'

        newname2='UVIS{}_'.format(mainccd_2)+el

        orig_order=np.arange(1, len(hdulist))
        new_order=np.zeros(len(orig_order), dtype=np.int)
        for i in xrange(len(orig_order)):
            new_order[i]=orig_order[i]

        new_order[0]=orig_order[3]
        new_order[1]=orig_order[4]
        new_order[2]=orig_order[5]
        new_order[3]=orig_order[0]
        new_order[4]=orig_order[1]       
        new_order[5]=orig_order[2]    
   
        print 'Original order of extensions\t',orig_order, 'in %s'%(newname1)
        print 'New order of extensions \t',new_order, 'in %s'%(newname2)

        newhdu=fits.HDUList(hdus=[bu[0]])
        for new_data in new_order:
            newhdu.append(hdulist[new_data])

        if onlyshow:
            print 'save %s, assigning MAINCCD to %s, FCNUM to %s'%(newname2, mainccd_2, field_nr)
            newhdu.close()
            hdulist.close()
        else:
            newhdu.writeto(newname2, clobber=True)
            fits.setval(newname2, 'MAINCCD', value=str(mainccd_2)) #CCD for image in extension 1 (UVIS)
            fits.setval(newname2, 'FCNUM', value=field_nr)
            
        """

In case one wants to split the image into two new images, the old one with newname1 (which can be changed from el to UVIS#_el) and the new one.
The advantage is that the new order of extensions in the new file is:
0 4 5 6 1 2 3 7 8 9 10 11 ...
The files now contain the same general information but in the one UVIS1 is in ext 1 in the other UVIS2

In [ ]:
    print 'assigned %i different fieldcenters'%(len(ra_u))

    #plotting image centers with mathplotlib
    for ra,dec in zip(ra_u, dec_u):
        plt.plot(ra,dec, 'o')
    plt.show()
    return ()

if done print out how many fcnums are given and plot them in different colors

In [ ]:
def change_name(field, nr):
    name_str=field+str(nr)
    return name_str

The most unnecessary thing ever, can be modified though to change the file name into more complicated stuff

In [ ]:
def check(ra1, ra2, dec1, dec2, radius):
    myaos=aos(ra1, ra2, dec1, dec2)
    return (myaos<=radius)

Calculates the angle of separation:
cos(y)=cos(a)+cos(b)-1
compares the result to the search radius, returns True if Radius is higher, False if not

In [ ]:
if __name__ == '__main__':
    args=get_opt()
    if not args.f:
        if args.survey==None:
            i=str(raw_input('No -survey input, continue? (y/n): \t'))
            if i in ['q', 'exit', 'quit', 'n']:
                sys.exit('exiting...')
            else:
                pass

One is asked to fill in a survey name (e.g. 30dor or PHAT, the keyword ill be set and used in rename_jhein.pl) This isfor giving the images one belonging name independent of their PID or FILTER. Can also be done by hand 
add_keyword.py SURVEY [VALUE]

In [ ]:
    scaling=0.04 #arcsec/pix
    scaling/=3600. #deg/pix
    direc=args.dir

    filenames=[]
    for x in os.listdir(direc):
        if x[-5:]=='.fits':
            filenames.append(x)
    filenames=sorted(filenames)
    rename(direc, filenames, scaling, radius=args.radius, onlyshow=args.onlyshow)

Scaling for HST WFC3 UVIS images is 0.04 arcsec/pix.
It takes all files ending on .fits